# Command Quick Reference
---

## Loading cdapython package


In [1]:
from cdapython import Q, columns, unique_terms, query

## cdapython version

In [13]:
import cdapython
print(Q.get_version())

2022.6.28


## columns()

Show all searchable columns in the CDA


**Usage:**
`columns(<optional args>)`

**Options:**
- `limit = <number>`: Changes the number of columns returned. Default = 100

**Attributes:**  
- `.to_dataframe()`: Returns results as a dataframe
- `.to_list()`: Returns results as a list
    
**Returns:** object containing column names


## unique_terms()

For a given column show all unique terms.

**Usage:** `unique_terms(<column_name>, <optional args>)`

**Options:**
- `limit = <number>`: Changes the number of terms returned. Default = 100
- `system= <data source>`: Returns only terms that belong to the specified datasource. Currently `GDC`, `IDC`, or `PDC`
- `show_counts = <True/False>`: Returns terms as a dataframe that counts how often each term appears. Must be used with `.to_dataframe()`

**Attributes:**  
- `.to_dataframe()`: Returns results as a dataframe.
- `.to_list()`: Returns results as a list.
    
**Returns:** searchable terms for a given column

## to_list()

**Usage:** `columns().to_list(<optional args>)`

**Options:**
- `filters = <string>`: Only returns columns that contain <string>
- `exact = <True/False>`: Alters the behavior of `filters` to only return columns that are an exact match to <string>

**Returns:** results as a list 

## to_dataframe()

**Usage:** `columns().to_dataframe()`

**Returns:** results as a list 

## Q()

Executes this query on the public CDA server.

**Usage:** `<result> = Q(<column> <operator> <term>)`

**Where:**
- `<result>` is any user created variable
- `<column>` is a value from `columns()`
- `<operator>` is a valid comparison operator
- `<term>` is a numeric/boolean/unique value from `unique_terms()`

**Returns:** cda-python Q data type
    
    

### Q().run()

Retrieves results of a `Q()` query.

**Usage:** `<result>.run(<optional args>)`

**Where:**
- `<result>` is a user created variable containing an executed `Q()` query

In [5]:
help(Q.run)

Help on function run in module cdapython.Q:

run(self: ~TQ, offset: int = 0, limit: int = 100, version: Union[str, NoneType] = None, host: Union[str, NoneType] = None, dry_run: bool = False, table: Union[str, NoneType] = None, async_call: bool = False, verify: Union[bool, NoneType] = None, verbose: Union[bool, NoneType] = True, filter: Union[str, NoneType] = None, flatten: Union[bool, NoneType] = False, format: str = 'json', show_sql: Union[bool, NoneType] = False) -> Union[cdapython.results.result.Result, cda_client.model.query_created_data.QueryCreatedData, multiprocessing.pool.ApplyResult]
    _summary_
    
    Args:
        offset (int, optional): _description_. Defaults to 0.
        limit (int, optional): _description_. Defaults to 100.
        version (Optional[str], optional): _description_. Defaults to None.
        host (Optional[str], optional): _description_. Defaults to None.
        dry_run (bool, optional): _description_. Defaults to False.
        table (Optional[str],

### Q().run().join_as_str()

Returns a single column of data as a string

**Usage:** mycolumn = `<result>.subject.run(filter='<column name>').join_as_str(key="<column name>",delimiter=",")

**Where:**
- `<result>` is a user created variable containing an executed `Q()` query
- `<column name>` is the column of desired information

The resulting string variable can be directly used as input to a new `Q()` query. This example would retrieve all the files for the subjects that met the conditions of the query above:

`Q(f'<column name> IN ({mycolumn})').file.run()`


### Q().count()

Retrieves summary counts of a `Q()` query.

**Usage:** `<result>.count()`

**Where:**
- `<result>` is a user created variable containing an executed `Q()` query

### Valid `Q()` comparison operators 

`Q` uses the following operators:,

- `=` : Equals
- `!=` : Not Equal
- `OR`
- `AND`
- `IN` and `NOT IN`\
- `%`: pattern matching a wildcard
- `IS` and `IS NOT`
- `>`, `>`, `>=`, `<=`: Greater and Less than

Check out the [Operators guide](../../Examples/Operators/#Equals:-=) for more info

### filter

Returns only the requested columns in a result. Can be used to return fewer columns from an endpoint, or to add columns that would not normally be returned to an endpoint result. In either case all desired columns must be listed.

Return single column:

`Usage: myquery.run(filter="<column name>")`

Return multiple columns:

`Usage: myquery.run(filter="<column name> <column name> <column name>")`

To add large numbers of columns to a result, it may be easier to get data from two endpoints and merge them, see an example in the [Cohort Building workflow](../../Examples/BuildingACohort/#merging-results-across-endpoints)


### Pagination

By default, `Q` only returns the first 100 rows of data for each search. This keeps search fast while still allowing you to preview the data. Once you have a query that returns the data you want, you'll use the `paginator` function to return all the data as a dataframe or list.

#### Usage:

##### To write to a dataframe
``` python

import pandas as pd
myquery = Q(<your search parameters>) 
    
mydataframe = pd.DataFrame() #create an emptydataframe
for i in myquery.subject.run().paginator(to_df=True): #loops through entire result, appending each page to your dataframe
    mydataframe = pd.concat([mydataframe, i])
```

##### To write to a list

``` python
import pandas as pd
myquery = Q(<your search parameters>) 

mylist = []
for i in myquery.subject.run().paginator(to_list=True):
    mylist.extend(i)
```